<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-of-different-data-sources-into-dataframe" data-toc-modified-id="Import-of-different-data-sources-into-dataframe-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import of different data sources into dataframe</a></span><ul class="toc-item"><li><span><a href="#weather-data" data-toc-modified-id="weather-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>weather data</a></span></li><li><span><a href="#traffic-data" data-toc-modified-id="traffic-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>traffic data</a></span></li><li><span><a href="#Merge-MIV-and-Weather-Dataset" data-toc-modified-id="Merge-MIV-and-Weather-Dataset-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Merge MIV and Weather Dataset</a></span></li><li><span><a href="#oil-price" data-toc-modified-id="oil-price-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>oil price</a></span></li></ul></li><li><span><a href="#Load-cleaned-data-to-DWH" data-toc-modified-id="Load-cleaned-data-to-DWH-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load cleaned data to DWH</a></span><ul class="toc-item"><li><span><a href="#Load-cleaned-oilprice-to-dwh" data-toc-modified-id="Load-cleaned-oilprice-to-dwh-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load cleaned oilprice to dwh</a></span></li><li><span><a href="#Load-cleaned-weather_traffic-to-dwh" data-toc-modified-id="Load-cleaned-weather_traffic-to-dwh-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load cleaned weather_traffic to dwh</a></span></li></ul></li></ul></div>

In [20]:
import pandas as pd
import psycopg2 as pg
from datetime import date, datetime
import time
from sqlalchemy import create_engine, text, inspect
import requests
import json
import datetime
import logging
import pytz
import warnings

In [21]:
warnings.filterwarnings('ignore')

## Import of different data sources into dataframe

### weather data

In [22]:
# DB credentials
ENDPOINT = 'trunks.cwh7dj4q8zyj.us-east-1.rds.amazonaws.com'
DB_NAME = 'trunks'
USERNAME = 'trunks'
PASSWORD = 'Corn5!!!'


# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'
    
# create sql alchemy engine
db = create_engine(conn_string)
dbconnect = db.connect()

Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 558, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



In [23]:
 # check existing tables in DB
insp = inspect(db)
existing_tables = insp.get_table_names()
print(existing_tables)

['weather_hourly_urls', 'weather_daily', 'weather_daily_urls', 'weather_hourly']


In [24]:
df_weather = pd.read_sql_table('weather_hourly', dbconnect) #hourly data (note: longer loading time)

In [25]:
#df_weather.head()

In [26]:
df_weather_aggr = df_weather[['Datum','Standort','Parameter','Einheit','Wert']]
df_weather_aggr.rename({'Datum': 'Datetime', 'Standort': 'Location', 'Einheit': 'Unit', 'Wert': 'Value'}, axis=1, inplace=True)

In [27]:
parameter_keep = ["RainDur","T"]
df_weather_aggr = df_weather_aggr[df_weather_aggr['Parameter'].isin(parameter_keep)]
#df_weather_aggr

In [28]:
df_weather_aggr.head()

,Datetime,Location,Parameter,Unit,Value
0,2017-01-31T16:00+0100,Zch_Rosengartenstrasse,RainDur,min,59.48
1,2017-01-31T16:00+0100,Zch_Rosengartenstrasse,T,°C,7.95
7,2017-01-31T16:00+0100,Zch_Schimmelstrasse,RainDur,min,60
8,2017-01-31T16:00+0100,Zch_Schimmelstrasse,T,°C,None
13,2017-01-31T16:00+0100,Zch_Stampfenbachstrasse,RainDur,min,60


In [29]:
#The Location name is different to the MIV Data so wie harmonize it
df_weather_aggr['Location'] = df_weather_aggr['Location'].str.replace('Zch_','',regex=True)

In [30]:
df_weather_aggr['Value'] = df_weather_aggr['Value'].astype(float)
df_weather_aggr['Datetime'] = pd.to_datetime(df_weather_aggr['Datetime'],utc=False)
df_weather_aggr['DatetimeUTC'] = pd.to_datetime(df_weather_aggr['Datetime'],utc=True)
#df_weather_aggr.info()
#df_weather_aggr

In [31]:
del df_weather_aggr['Datetime']
df_weather_aggr = df_weather_aggr[["DatetimeUTC", "Location", "Parameter","Value","Unit"]]

In [32]:
df_weather_aggr.sort_values("DatetimeUTC")

,DatetimeUTC,Location,Parameter,Value,Unit
2460421,1992-06-30 22:00:00+00:00,Stampfenbachstrasse,T,18.93,°C
2460428,1992-06-30 23:00:00+00:00,Stampfenbachstrasse,T,18.60,°C
2460435,1992-07-01 00:00:00+00:00,Stampfenbachstrasse,T,18.24,°C
2460442,1992-07-01 01:00:00+00:00,Stampfenbachstrasse,T,17.93,°C
2460449,1992-07-01 02:00:00+00:00,Stampfenbachstrasse,T,17.75,°C
...,...,...,...,...,...
2549663,2022-04-21 10:00:00+00:00,Stampfenbachstrasse,RainDur,0.00,min
2549668,2022-04-21 10:00:00+00:00,Schimmelstrasse,T,14.72,°C
2549671,2022-04-21 10:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
2549675,2022-04-21 10:00:00+00:00,Rosengartenstrasse,T,13.75,°C


In [33]:
df_weather_aggr = df_weather_aggr[df_weather_aggr["DatetimeUTC"].dt.strftime('%Y') >= '2013'] #only from 2013 upwards relevant

In [34]:
df_weather_aggr.sort_values("DatetimeUTC")

,DatetimeUTC,Location,Parameter,Value,Unit
684440,2013-01-01 00:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
684441,2013-01-01 00:00:00+00:00,Schimmelstrasse,T,2.04,°C
684446,2013-01-01 00:00:00+00:00,Stampfenbachstrasse,RainDur,0.00,min
684447,2013-01-01 00:00:00+00:00,Stampfenbachstrasse,T,2.06,°C
684454,2013-01-01 01:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
...,...,...,...,...,...
2549663,2022-04-21 10:00:00+00:00,Stampfenbachstrasse,RainDur,0.00,min
2549668,2022-04-21 10:00:00+00:00,Schimmelstrasse,T,14.72,°C
2549671,2022-04-21 10:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
2549675,2022-04-21 10:00:00+00:00,Rosengartenstrasse,T,13.75,°C


### traffic data

In [35]:
# DB credentials
ENDPOINT = 'traffic.cqznezykh98v.us-east-1.rds.amazonaws.com'
DB_NAME = 'traffic'
USERNAME = 'admin_user'
PASSWORD = 'KAjUGqLSwNhTrf6m58Ap'


# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'
    
# create sql alchemy engine
db = create_engine(conn_string)
dbconnect = db.connect()

In [36]:
 # check existing tables in DB
insp = inspect(db)
existing_tables = insp.get_table_names()
print(existing_tables)

['miv_hourly']


In [37]:
df_traffic = pd.read_sql_table('miv_hourly', dbconnect) #hourly data
df_traffic.head()

,MSID,ZSID,ZSName,Achse,HNr,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,MessungDatZeit,AnzFahrzeuge,AnzFahrzeugeStatus
0,Z038M001,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,2682070.5,1249909.1,Bucheggplatz,407,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,2022-01-01T00:00:00,581.0,Gemessen
1,Z038M001,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,2682070.5,1249909.1,Bucheggplatz,407,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,2022-01-01T01:00:00,1011.0,Gemessen
2,Z038M001,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,2682070.5,1249909.1,Bucheggplatz,407,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,2022-01-01T02:00:00,808.0,Gemessen
3,Z038M001,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,2682070.5,1249909.1,Bucheggplatz,407,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,2022-01-01T03:00:00,596.0,Gemessen
4,Z038M001,Z038,Rosengartenstrasse (Nordstrasse),Rosengartenstrasse,23,2682070.5,1249909.1,Bucheggplatz,407,Nordstr/Röschibachsteig/Rosengartenbrücke,2,14,15,2022-01-01T04:00:00,483.0,Gemessen


In [38]:
#df_traffic["Achse"].unique()
#df_traffic["Richtung"].unique()
#df_traffic["MSID"].unique()
#df_traffic["AnzFahrzeugeStatus"].unique()
#len(df_traffic[df_traffic["AnzFahrzeugeStatus"]=='Imputiert'])
#len(df_traffic[df_traffic["AnzFahrzeugeStatus"]=='Gemessen'])

In [39]:
df_traffic_aggr = df_traffic.groupby(['Achse','MessungDatZeit']).agg({'AnzFahrzeuge':'sum'}).reset_index()
#df_traffic_aggr.head()

In [40]:
df_traffic_aggr['Parameter'] = 'MIV' #add to have same structure like weather data
df_traffic_aggr['Unit'] = 'Count'  #add to have same structure like weather data
df_traffic_aggr.rename({'Achse': 'Location', 'MessungDatZeit': 'Datetime', 'AnzFahrzeuge': 'Value'}, axis=1, inplace=True)
#df_traffic_aggr.head()

In [41]:
df_traffic_aggr['Datetime'] = pd.to_datetime(df_traffic_aggr['Datetime'],utc=False)
df_traffic_aggr['Value'] = df_traffic_aggr['Value'].astype(float)

In [42]:
#pytz.all_timezones

datetimeUTC = []
notvaliddates = []
validdates = []

for index, row in df_traffic_aggr.iterrows():
    try:
        #print(str(row['Datetime']))
        local_time = pytz.timezone('Europe/Zurich')
        naive_datetime = datetime.datetime.strptime((str(row['Datetime'])), "%Y-%m-%d %H:%M:%S") #Convert to naive datetime object
        local_datetime = local_time.localize(naive_datetime, is_dst=True) #Update naive datetime object with local timezone
        y = local_datetime.astimezone(pytz.utc) #Convert to UTC
        #print(y)
        datetimeUTC.append(y)
        validdates.append(row['Datetime'])
        
    except:
        notvaliddates.append(row['Datetime'])

In [43]:
notvaliddates # if we set is_dest to "true" this must be empty

[]

In [44]:
df_traffic_aggr = df_traffic_aggr[df_traffic_aggr["Datetime"].isin(validdates)]

In [45]:
#quality check before binding

In [46]:
len(df_traffic_aggr) == len(datetimeUTC) == len(validdates) # must be true

True

In [18]:
df_datetimeUTC = pd.DataFrame (datetimeUTC, columns = ['DatetimeUTC'])

In [47]:
df_datetimeUTC.iloc[5000]
df_traffic_aggr.iloc[5000]

Location      Rosengartenstrasse
Datetime     2012-07-27 09:00:00
Value                     1088.0
Parameter                    MIV
Unit                       Count
Name: 5000, dtype: object

In [48]:
df_traffic_aggr = pd.concat([df_traffic_aggr, df_datetimeUTC], axis=1)

In [49]:
df_traffic_aggr.head(5000)

,Location,Datetime,Value,Parameter,Unit,DatetimeUTC
0,Rosengartenstrasse,2012-01-01 00:00:00,1025.0,MIV,Count,2011-12-31 23:00:00+00:00
1,Rosengartenstrasse,2012-01-01 01:00:00,2006.0,MIV,Count,2012-01-01 00:00:00+00:00
2,Rosengartenstrasse,2012-01-01 02:00:00,1726.0,MIV,Count,2012-01-01 01:00:00+00:00
3,Rosengartenstrasse,2012-01-01 03:00:00,1384.0,MIV,Count,2012-01-01 02:00:00+00:00
4,Rosengartenstrasse,2012-01-01 04:00:00,1320.0,MIV,Count,2012-01-01 03:00:00+00:00
...,...,...,...,...,...,...
4995,Rosengartenstrasse,2012-07-27 04:00:00,334.0,MIV,Count,2012-07-27 02:00:00+00:00
4996,Rosengartenstrasse,2012-07-27 05:00:00,744.0,MIV,Count,2012-07-27 03:00:00+00:00
4997,Rosengartenstrasse,2012-07-27 06:00:00,1276.0,MIV,Count,2012-07-27 04:00:00+00:00
4998,Rosengartenstrasse,2012-07-27 07:00:00,1250.0,MIV,Count,2012-07-27 05:00:00+00:00


We can now see that the DatetimeUTC is depoending on summer or wintertime beween -1 and -2 hours. We normalized now alle dates to UTC time to save them into the database the same way.

In [50]:
#del df_traffic_aggr['Datetime']
df_traffic_aggr = df_traffic_aggr[["DatetimeUTC", "Location", "Parameter","Value","Unit"]]

In [51]:
df_traffic_aggr = df_traffic_aggr[df_traffic_aggr["DatetimeUTC"].dt.strftime('%Y') >= '2013'] #only from 2013 upwards relevant

In [52]:
df_traffic_aggr.sort_values("DatetimeUTC")

,DatetimeUTC,Location,Parameter,Value,Unit
8784,2013-01-01 00:00:00+00:00,Rosengartenstrasse,MIV,2090.0,Count
99133,2013-01-01 00:00:00+00:00,Schimmelstrasse,MIV,1189.0,Count
189482,2013-01-01 00:00:00+00:00,Stampfenbachstrasse,MIV,669.0,Count
8785,2013-01-01 01:00:00+00:00,Rosengartenstrasse,MIV,1670.0,Count
99134,2013-01-01 01:00:00+00:00,Schimmelstrasse,MIV,878.0,Count
...,...,...,...,...,...
90347,2022-04-24 20:00:00+00:00,Rosengartenstrasse,MIV,2652.0,Count
180696,2022-04-24 20:00:00+00:00,Schimmelstrasse,MIV,1198.0,Count
180697,2022-04-24 21:00:00+00:00,Schimmelstrasse,MIV,998.0,Count
90348,2022-04-24 21:00:00+00:00,Rosengartenstrasse,MIV,2149.0,Count


### Merge MIV and Weather Dataset

In [53]:
df_weather_aggr.reset_index(drop=True, inplace=True)
df_traffic_aggr.reset_index(drop=True, inplace=True)
df_weather_traffic_clean = pd.concat([df_weather_aggr,df_traffic_aggr],axis=0)
df_weather_traffic_clean = df_weather_traffic_clean.sort_values("Parameter",ascending = False)
df_weather_traffic_clean = df_weather_traffic_clean.sort_values("Location",ascending = False)
df_weather_traffic_clean = df_weather_traffic_clean.sort_values("DatetimeUTC",ascending = False)

In [54]:
df_weather_traffic_clean.head(1000)
#df_weather_traffic_clean.info()

,DatetimeUTC,Location,Parameter,Value,Unit
244694,2022-04-24 21:00:00+00:00,Stampfenbachstrasse,MIV,191.00,Count
163129,2022-04-24 21:00:00+00:00,Schimmelstrasse,MIV,998.00,Count
81564,2022-04-24 21:00:00+00:00,Rosengartenstrasse,MIV,2149.00,Count
244693,2022-04-24 20:00:00+00:00,Stampfenbachstrasse,MIV,258.00,Count
163128,2022-04-24 20:00:00+00:00,Schimmelstrasse,MIV,1198.00,Count
...,...,...,...,...,...
260573,2022-04-18 00:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
260564,2022-04-17 23:00:00+00:00,Stampfenbachstrasse,T,8.09,°C
260567,2022-04-17 23:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
260566,2022-04-17 23:00:00+00:00,Schimmelstrasse,T,8.49,°C


### oil price

In [55]:
# DB credentials
ENDPOINT = 'datalake.cknmu1bvrxjg.us-east-1.rds.amazonaws.com'
DB_NAME = 'datalake'
USERNAME = 'muser'
PASSWORD = 'datalake'


# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'
    
# create sql alchemy engine
db = create_engine(conn_string)
dbconnect = db.connect()

In [39]:
 # check existing tables in DB
insp = inspect(db)
existing_tables = insp.get_table_names()
print(existing_tables)

['oilprice', 'metadata_oil', 'news_oil']


In [40]:
df_oil = pd.read_sql_table('oilprice', dbconnect) #daily data

In [41]:
df_oil.head()
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3652 non-null   datetime64[ns]
 1   open    3652 non-null   float64       
 2   high    3652 non-null   float64       
 3   low     3652 non-null   float64       
 4   close   3652 non-null   float64       
 5   volume  3652 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 171.3 KB


In [42]:
df_oil_aggr = df_oil[['date','open']]

In [43]:
df_oil_aggr.rename({'date': 'Date', 'open': 'Value'}, axis=1, inplace=True)
df_oil_aggr['Parameter'] = 'Price' #add to have same structure like weather data
df_oil_aggr['Unit'] = '$'  #add to have same structure like weather and Traffic data
df_oil_aggr = df_oil_aggr[["Date", "Parameter","Value","Unit"]]
df_oil_aggr

,Date,Parameter,Value,Unit
0,2007-07-30,Price,75.849998,$
1,2007-07-31,Price,75.699997,$
2,2007-08-01,Price,77.000000,$
3,2007-08-02,Price,75.220001,$
4,2007-08-03,Price,75.389999,$
...,...,...,...,...
3647,2022-04-18,Price,111.889999,$
3648,2022-04-19,Price,112.650002,$
3649,2022-04-20,Price,107.720001,$
3650,2022-04-21,Price,107.010002,$


In [44]:
df_oil_clean = df_oil_aggr

## Load cleaned data to DWH

### Load cleaned oilprice to dwh

In [45]:
# DB credentials
ENDPOINT = 'lakehousedwh.cejfhcomb8d8.us-east-1.rds.amazonaws.com'
DB_NAME = 'lakehousedwh'
USERNAME = 'trunks'
PASSWORD = 'Corn5!!!'


# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'
    
# create sql alchemy engine
engine = create_engine(conn_string)

In [46]:
df_oil_clean.to_sql('oil_stage', engine, if_exists='replace',index=False, method='multi')

### Load cleaned weather_traffic to dwh

In [47]:
#df_weather_traffic_small = df_weather_traffic_clean.head(100)
#df_weather_traffic_medium = df_weather_traffic_clean.head(1000)
#df_weather_traffic_big = df_weather_traffic_clean.head(100000)

In [48]:
df_weather_traffic_clean.to_sql('weather_traffic_stage', engine, if_exists='replace',index=False, method='multi',chunksize=5000)